# Our validation - ideas + results confirmation

In [26]:
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, QuantileTransformer

In [24]:
test_df = pd.read_csv('C:\\Users\\rogal\\UczenieMaszynowe\\Projekt2\\post_feature_engineering_test_data.csv')
# waiting for train df 

In [25]:
test_df

,seconds_elapsed,location_bearingAccuracy,location_speedAccuracy,location_verticalAccuracy,location_horizontalAccuracy,location_speed,location_bearing,location_altitude,location_longitude,location_latitude,...,Bandpass_Gyroscope_z,avg_steps_per_minute,bearing_east,bearing_north,bearing_south,bearing_west,combined_orientation,Total_Motion_Intensity,Gravity_Magnetometer_y_interaction,Gravity_Orientation_pitch_ratio
0,351.988000,21.500001,2.6,1.035250,3.900,0.395205,92.127159,68.599998,3.095426,50.688277,...,0.004951,0.000000,True,False,False,False,-0.023727,18.002959,-14.952563,-9.790013
1,78.412056,0.000000,0.0,6.363528,6.699,2.084059,315.506165,89.200005,3.139791,50.684868,...,-0.286395,84.170730,False,False,False,True,-0.865642,50.304606,-380.821190,-8.482901
2,771.097949,0.000000,0.0,1.055240,7.751,0.717536,142.522339,90.000000,3.152066,50.681880,...,-0.187927,106.134376,True,False,False,False,-1.234499,28.346709,-250.688916,-7.311745
3,328.574522,0.000000,0.0,1.228584,7.994,1.284345,102.234695,87.300003,3.146482,50.685465,...,-0.406713,107.738116,True,False,False,False,-0.698521,42.789231,-360.973372,-7.468350
4,271.752000,5.700000,1.0,30.814772,3.900,7.353899,274.938171,82.500000,3.098769,50.686902,...,-0.816570,104.433454,False,False,False,True,-0.135499,15.919612,-31.234479,-9.715313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689,466.098125,0.000000,0.0,15.400000,3.658,0.001336,0.000000,68.900000,3.111234,50.681033,...,0.762913,98.219661,False,True,False,False,-0.640401,28.407376,-30.133030,-9.717622
2690,951.271000,21.500001,3.9,1.469109,5.285,0.113767,335.953064,89.599998,3.152528,50.681563,...,0.054776,93.222646,False,False,False,True,0.505617,46.420563,-415.136369,-7.416683
2691,933.701806,0.000000,0.0,14.500000,6.781,3.123722,20.089300,71.900000,3.126475,50.675735,...,-0.349545,100.824481,False,True,False,False,-0.653496,37.788792,-160.888656,-9.410930
2692,314.854788,0.000000,0.0,19.700001,6.610,4.457863,322.790863,57.800000,3.119078,50.674101,...,-0.247453,102.523453,False,False,False,True,-0.737389,47.456021,-253.501862,-9.258339


In [ ]:
# budowa robi Quantile transformer
transformer = QuantileTransformer(output_distribution='normal')
df_transformed = pd.DataFrame(transformer.fit_transform(test_df), columns=test_df.columns)

#spróbujmy równocześnie StandardScaler 
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform())

### Podpowiedź do PCA:
zamiast

In [19]:
pca = PCA(n_components=len(test_df.columns))
pca.fit(test_df)

PCA(n_components=64)

In [23]:
explained_variance = pca.explained_variance_ratio_
print(explained_variance)
cumulative_explained_variance = explained_variance.cumsum()
threshold = 0.90
num_components = np.argmax(cumulative_explained_variance >= threshold) + 1
print(num_components)

[0.99979887]
1


można użyć funkcji PCA:

In [22]:
pca = PCA(0.9)
pca.fit(test_df)
print(pca.n_components_)

1


### Pomysł do wizualizacji klastrów PCA

Grupa budująca w wizualizacji klastrów skupiła się na n_clusters = 4 bo wynki silhouette były najlepsze. Zobaczmy ile klastrów sugeruje elbow plot i czy faktycznie 4: